In [52]:
import numpy as np
import pandas as pd
import math
import os
from typing import Tuple, List

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [53]:
#Reading file

def parse_xy_data(filepath, delimiter=None):
    """
    Reads a CSV-like file with x, y data and returns a list of (x, y) tuples.

    This function is designed to handle different delimiters (like ';' or ' ')
    and assumes that commas (',') are used as decimal separators, based on
    the provided image.

    Args:
        filepath (str): The path to the data file.
        delimiter (str, optional): The column delimiter (e.g., ';', ' '). 
                                   If None, the function will try to 
                                   auto-detect it.

    Returns:
        list: A list of (x, y) float tuples.
              Returns an empty list if the file cannot be read or is empty.
    """
    data_points = []
    detected_delimiter = delimiter
    
    # --- 1. Delimiter Sniffing (if not provided) ---
    if detected_delimiter is None:
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                # Read the first non-empty line to guess
                first_line = ""
                for line in f:
                    first_line = line.strip()
                    if first_line:
                        break
                
                if ';' in first_line:
                    detected_delimiter = ';'
                elif ' ' in first_line:
                    # Check if it's likely a space delimiter
                    parts = re.split(r'\s+', first_line)
                    if len(parts) == 2:
                        try:
                            # Try to parse to see if it makes sense
                            float(parts[0].replace(',', '.'))
                            float(parts[1].replace(',', '.'))
                            detected_delimiter = ' '
                        except (ValueError, IndexError):
                             # Not a valid 2-column space-delimited float line
                             pass
                
                if detected_delimiter is None and ',' in first_line:
                    # Comma is the last guess, as it's ambiguous with decimal
                    detected_delimiter = ','
                
                if detected_delimiter is None:
                    # Final fallback based on your image
                    print("Warning: Could not auto-detect delimiter. Falling back to ';'.")
                    detected_delimiter = ';'
        except Exception as e:
            print(f"Error opening/reading file for sniffing: {e}")
            return [] # Return empty list on error
    
    print(f"Using delimiter: '{detected_delimiter}'")

    # --- 2. File Parsing ---
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line_number, line in enumerate(f, 1):
                line = line.strip()
                if not line or line.startswith('#'):
                    continue # Skip empty lines or comment lines

                # Split the line by the detected delimiter
                if detected_delimiter == ' ':
                    # Use regex split for spaces to handle multiple spaces
                    parts = re.split(r'\s+', line)
                else:
                    parts = line.split(detected_delimiter)

                # Ensure we have exactly two columns
                if len(parts) == 2:
                    x_str, y_str = parts
                    
                    try:
                        # KEY STEP: Replace comma with dot for float conversion
                        x_val = float(x_str.strip().replace(',', '.'))
                        y_val = float(y_str.strip().replace(',', '.'))
                        data_points.append((x_val, y_val))
                    except ValueError as e:
                        # Warn if conversion to float fails
                        print(f"Warning: Could not parse numbers on line {line_number}: '{line}'. Error: {e}")
                else:
                    # Warn if the line doesn't have exactly two parts
                    print(f"Warning: Skipping malformed line {line_number}: '{line}'. Expected 2 columns, found {len(parts)}")
    
    except FileNotFoundError:
        print(f"Error: File not found at '{filepath}'")
        return []
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return []

    return data_points

In [54]:
print("--- Example: Reading '19data.csv' ---")
    
# You would replace '19data.csv' with the path to your actual file
file_to_read = '20251GKtest2.csv' 
    
# Check if the file exists before trying to read it
if os.path.exists(file_to_read):
    # Call the function to parse the file.
    # It will try to auto-detect the delimiter.
    all_points = parse_xy_data(file_to_read)
        
    if all_points:
        print(f"Successfully parsed {len(all_points)} data points:")
        df_input = pd.DataFrame(all_points, columns=['x', 'y'])
        print(df_input.to_string(index=False))
    else:
        print(f"Could not parse any data points from '{file_to_read}'.")
        print("Please check the file format and warnings above.")
            
else:
    print(f"Error: The file '{file_to_read}' was not found.")
    print("Please create this file or change 'file_to_read' variable")
    print("to point to your existing data file.")

--- Example: Reading '19data.csv' ---
Using delimiter: ';'
Successfully parsed 104 data points:
     x       y
 1.000  8.6363
 1.115  6.6105
 1.230  5.2241
 1.345  4.2221
 1.460  3.4642
 1.575  2.8724
 1.690  2.3975
 1.805  2.0098
 1.920  1.6898
 2.035  1.4218
 2.150  1.1946
 2.265  1.0018
 2.380  0.8401
 2.495  0.7006
 2.610  0.5865
 2.725  0.4809
 2.840  0.3958
 2.955  0.3245
 3.070  0.2665
 3.185  0.2208
 3.300  0.1753
 3.415  0.1397
 3.530  0.1129
 3.645  0.0867
 3.760  0.0725
 3.875  0.0588
 3.990  0.0426
 4.105  0.0328
 4.220  0.0239
 4.335  0.0184
 4.450  0.0157
 4.565  0.0116
 4.680  0.0095
 4.795  0.0074
 4.910  0.0056
 5.025  0.0042
 5.140  0.0058
 5.255  0.0073
 5.370  0.0020
 5.485  0.0043
 5.600  0.0081
 5.715  0.0007
 5.830  0.0024
 5.945  0.0045
 6.060  0.0026
 6.175  0.0084
 6.290  0.0097
 6.405  0.0040
 6.520  0.0082
 6.635  0.0058
 6.750  0.0022
 6.865  0.0052
 6.980  0.0010
 7.095  0.0013
 7.210  0.0051
 7.325  0.0007
 7.440  0.0002
 7.555  0.0011
 7.670  0.0008
 7.7

In [55]:
def get_points_in_range(points: List[Tuple[float, float]], 
                        x_start: float, 
                        x_end: float) -> List[Tuple[float, float]]:
    """
    Filters a list of (x, y) tuples to return only those within a 
    specified x-range [x_start, x_end].
    """
    subset_points = [
        (x, y) for (x, y) in points 
        if x_start <= x <= x_end
    ]
    return subset_points


In [56]:
# Select points for the second interval [9.7, 10.0]
# This interval is suitable for Newton's FORWARD method

x_start_demo = 2.150
x_end_demo = 2.955
    
monotonic_points_fwd = get_points_in_range(all_points, x_start_demo, x_end_demo)
    
print(f"Subset of points from x={x_start_demo} to x={x_end_demo}:")
df_subset = pd.DataFrame(monotonic_points_fwd, columns=['x (swap)', 'y (swap)'])
print(df_subset.to_string(index=False))

Subset of points from x=2.15 to x=2.955:
 x (swap)  y (swap)
    2.150    1.1946
    2.265    1.0018
    2.380    0.8401
    2.495    0.7006
    2.610    0.5865
    2.725    0.4809
    2.840    0.3958
    2.955    0.3245


In [57]:
def EvenDifference(points):
    """
    Calculates the finite difference table for evenly spaced nodes.
    
    Args:
        points (list of tuples): A list of (x, y) data points, sorted by x.

    Returns:
        pandas.DataFrame: The full difference table.
    """
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]
    n = len(y_values)

    # Internal calculation table
    diff_calc_table = np.full((n, n), np.nan)
    diff_calc_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_calc_table[i, j] = diff_calc_table[i+1, j-1] - diff_calc_table[i, j-1]

    # Format the output DataFrame
    data = {
        'x_i': x_values,
        'y_i': y_values
    }
    for j in range(1, n):
        # Pad with NaN to maintain table shape
        col_name = f'Order {j}'
        col_data = np.full(n, np.nan)
        col_data[:(n-j)] = diff_calc_table[:(n-j), j]
        data[col_name] = col_data
    
    df = pd.DataFrame(data)
    return df


# Bessel

## Algorithm

0. Conditions
* Used for **evenly spaced nodes**: $x_k = x_0 + kh$, $k \in \mathbb{Z}$.
* Requires an **even number of nodes** (which means an odd degree $n$).
* The formula is centered *between* two nodes, $x_0$ and $x_1$.

1. Finite Difference Table

Given $n+1$ nodes (where $n$ is odd). The formula is centered between $x_0$ and $x_1$.
A standard finite difference table is computed.

*(This is performed by the `EvenDifference(points)` function, which can be found in `Week6-Interpolation/pp2_Newton/pp2b_NewtonFixedGap.ipynb`)*

2. Coefficient Selection

The coefficients $C_i$ are selected from the difference table horizontally between the $y_0$ and $y_1$ rows:
* $C_0 = \frac{y_0 + y_1}{2}$
* $C_1 = \Delta y_0$
* $C_2 = \frac{\Delta^2 y_{-1} + \Delta^2 y_0}{2}$
* $C_3 = \Delta^3 y_{-1}$
* $C_4 = \frac{\Delta^4 y_{-2} + \Delta^4 y_{-1}}{2}$
* $C_5 = \Delta^5 y_{-2}$
* ...
* **General form:**
    * $C_{2k} = \frac{\Delta^{2k} y_{-k} + \Delta^{2k} y_{-(k-1)}}{2}$
    * $C_{2k+1} = \Delta^{2k+1} y_{-k}$

3. Polynomial Construction in $t$

The polynomial is constructed in the variable $t = \frac{x - x_0}{h}$. The formula is "centered" at $t=0.5$.
$P(t) = \sum_{i=0}^{n} D_i B_i(t)$

1.  **Calculate Main Coefficients $D_i$:**
    $D_i = \frac{C_i}{i!}$

2.  **Calculate Basis Polynomials $B_i(t)$:**
    * $B_0(t) = 1$
    * $B_1(t) = (t - \frac{1}{2})$
    * $B_2(t) = t(t - 1)$
    * $B_3(t) = B_2(t) \cdot (t - \frac{1}{2})$
    * $B_4(t) = B_2(t) \cdot (t + 1)(t - 2)$
    * $B_5(t) = B_4(t) \cdot (t - \frac{1}{2})$
    * ...
    * **Recursive form:**
        * $B_{2k}(t) = B_{2k-2}(t) \cdot (t + k - 1)(t - k)$
        * $B_{2k+1}(t) = B_{2k}(t) \cdot (t - \frac{1}{2})$

3.  **Compute Total Polynomial $P(t)$:**
    $P(t) = \sum_{i=0}^{n} N_i(t) = \sum_{i=0}^{n} D_i B_i(t)$
    * The coefficients of each $N_i(t)$ are summed by degree to find the final coefficients of $P(t) = a_0 + a_1 t + \dots + a_n t^n$.

4.  **Output:** Return the intermediate steps table and the final coefficient list $a_k$ for $P(t)$.

In [58]:
def bessel_interpolation(points, x0_index):
    """
    Constructs the Bessel interpolation polynomial in the variable t = (x - x0) / h.
    
    Args:
        points (list of tuples): List of (x, y) data points. Must be evenly spaced
                                 and have an even number of points.
        x0_index (int): The index of the first central node (x0).

    Returns:
        step_pd (pd.DataFrame): DataFrame showing the intermediate steps.
        coeff_pd (pd.DataFrame): DataFrame of the final polynomial coefficients for P(t).
    """
    
    n = len(points) - 1
    if (n + 1) % 2 != 0:
        raise ValueError("Bessel's formula requires an even number of nodes (odd degree n).")
    
    # --- 1. Generate Difference Table ---
    diff_table_df = EvenDifference(points)

    # --- 2. Select Coefficients (Bessel Path) ---
    C_coeffs = []
    for i in range(n + 1):
        order_col = f'Order {i}' if i > 0 else 'y_i'
        
        if i % 2 == 0: # Even order: C_{2k} = (d^{2k} y_{-k} + d^{2k} y_{-(k-1)}) / 2
            k = i // 2
            c1_idx = x0_index - k
            c2_idx = x0_index - k + 1
            if c1_idx < 0 or c2_idx >= len(points):
                raise IndexError(f"Cannot access indices {c1_idx}, {c2_idx} for order {i}.")
            
            c1 = diff_table_df[order_col].iloc[c1_idx]
            c2 = diff_table_df[order_col].iloc[c2_idx]
            C_coeffs.append((c1 + c2) / 2.0)
            
        else: # Odd order: C_{2k+1} = d^{2k+1} y_{-k}
            k = (i - 1) // 2
            c_idx = x0_index - k
            if c_idx < 0 or c_idx >= len(points):
                raise IndexError(f"Cannot access index {c_idx} for order {i}.")
            C_coeffs.append(diff_table_df[order_col].iloc[c_idx])

    # --- 3. Build Polynomial P(t) ---
    steps_data = []
    N_coeffs_total = np.zeros(n + 1, dtype=float)
    
    # B_i(t) basis polynomials
    B_coeffs_list = []
    
    # B_0(t) = 1
    B_0 = np.array([1.0])
    B_coeffs_list.append(B_0)
    
    # B_1(t) = (t - 0.5)
    B_1 = np.array([-0.5, 1.0]) # [const, t]
    B_coeffs_list.append(B_1)

    for i in range(2, n + 1):
        if i % 2 == 0: # Even: B_{2k} = B_{2k-2} * (t+k-1)(t-k)
            k = i // 2
            # (t - k)
            term1 = np.array([-k, 1.0])
            # (t + k - 1)
            term2 = np.array([k - 1, 1.0])
            # (t^2 - t - k(k-1))
            new_factor = np.convolve(term1, term2)
            # B_{2k-2} is at index i-2
            B_i = np.convolve(B_coeffs_list[i-2], new_factor)
        else: # Odd: B_{2k+1} = B_{2k} * (t - 0.5)
            # B_{2k} is at index i-1
            B_i = np.convolve(B_coeffs_list[i-1], [-0.5, 1.0])
        
        B_coeffs_list.append(B_i)

    # Calculate final polynomial
    for i in range(n + 1):
        D_i = C_coeffs[i] / math.factorial(i)
        B_i = B_coeffs_list[i]
        
        # N_i(t) = D_i * B_i(t)
        Ni_coeffs = D_i * B_i
        
        # Add to total polynomial (pad with zeros)
        N_coeffs_total[:len(Ni_coeffs)] += Ni_coeffs

        # Store intermediate steps for printing
        steps_data.append({
            'i': i,
            'Difference Coeff (C_i)': C_coeffs[i],
            'D_i = C_i / i!': D_i,
            'B_i(t) Coeffs (low->high)': B_i.tolist(),
            'N_i(t) Coeffs (low->high)': Ni_coeffs.tolist()
        })

    step_pd = pd.DataFrame(steps_data)
    coeff_pd = pd.DataFrame({
        'Degree (t)': np.arange(n + 1),
        'Coeff': N_coeffs_total
    })

    return step_pd, coeff_pd

## Result

In [59]:
# 1. Define the data points from the image
points = monotonic_points_fwd

# 2. Set the central node index
# 9 points (indices 0-8), the center is index 4
x0_index = 3
x0_val = (points[x0_index][0] + points[x0_index + 1][0])/2
h = points[1][0] - points[0][0]

In [60]:
print("--- Generated Finite Difference Table ---")
df = EvenDifference(points)

df.style

--- Generated Finite Difference Table ---


,x_i,y_i,Order 1,Order 2,Order 3,Order 4,Order 5,Order 6,Order 7
0,2.150000,1.194600,-0.192800,0.031100,-0.008900,0.012100,-0.032200,0.081200,-0.177800
1,2.265000,1.001800,-0.161700,0.022200,0.003200,-0.020100,0.049000,-0.096600,nan
2,2.380000,0.840100,-0.139500,0.025400,-0.016900,0.028900,-0.047600,nan,nan
3,2.495000,0.700600,-0.114100,0.008500,0.012000,-0.018700,nan,nan,nan
4,2.610000,0.586500,-0.105600,0.020500,-0.006700,nan,nan,nan,nan
5,2.725000,0.480900,-0.085100,0.013800,nan,nan,nan,nan,nan
6,2.840000,0.395800,-0.071300,nan,nan,nan,nan,nan,nan
7,2.955000,0.324500,nan,nan,nan,nan,nan,nan,nan


In [61]:
# 3. Calculate the Bessel polynomial
step_df, final_coeff_df = bessel_interpolation(points, x0_index)

print("\n--- Polynomial Construction Steps (Bessel) ---")
step_df.style


--- Polynomial Construction Steps (Bessel) ---


,i,Difference Coeff (C_i),D_i = C_i / i!,B_i(t) Coeffs (low->high),N_i(t) Coeffs (low->high)
0,0,0.643550,0.643550,[1.0],[0.6435500000000001]
1,1,-0.114100,-0.114100,"[-0.5, 1.0]","[0.05704999999999999, -0.11409999999999998]"
2,2,0.016950,0.008475,"[0.0, -1.0, 1.0]","[0.0, -0.008474999999999983, 0.008474999999999983]"
3,3,-0.016900,-0.002817,"[0.0, 0.5, -1.5, 1.0]","[-0.0, -0.0014083333333333354, 0.0042250000000000065, -0.002816666666666671]"
4,4,0.004400,0.000183,"[0.0, 2.0, -1.0, -2.0, 1.0]","[0.0, 0.00036666666666667486, -0.00018333333333333743, -0.00036666666666667486, 0.00018333333333333743]"
5,5,0.049000,0.000408,"[0.0, -1.0, 2.5, 0.0, -2.5, 1.0]","[0.0, -0.00040833333333333325, 0.0010208333333333332, 0.0, -0.0010208333333333332, 0.00040833333333333325]"
6,6,-0.007700,-0.000011,"[0.0, -12.0, 4.0, 15.0, -5.0, -3.0, 1.0]","[-0.0, 0.00012833333333333723, -4.277777777777908e-05, -0.00016041666666667155, 5.347222222222385e-05, 3.208333333333431e-05, -1.069444444444477e-05]"
7,7,-0.177800,-0.000035,"[0.0, 6.0, -14.0, -3.5, 17.5, -3.5, -3.5, 1.0]","[-0.0, -0.00021166666666666643, 0.0004938888888888883, 0.00012347222222222208, -0.0006173611111111104, 0.00012347222222222208, 0.00012347222222222208, -3.527777777777774e-05]"


In [62]:
final_coeff_df.style

,Degree (t),Coeff
0,0,0.700600
1,1,-0.124108
2,2,0.013989
3,3,-0.003220
4,4,-0.001401
5,5,0.000564
6,6,0.000113
7,7,-0.000035


## Further Testing

In [63]:
#Horner Test
def synthetic_division(a, c):
    """
    Perform synthetic division for polynomial p(x) with coefficients a,
    evaluated at x = c.

    Parameters:
        a (list[float]): coefficients of p(x) from highest to lowest degree
        c (float): the value to evaluate p(c)

    Returns:
        df (pd.DataFrame): table with columns [i, a_i, b_i*c, b_i]
        p_c (float): value of p(c)
        q_coeff (list[float]): coefficients of q(x) = (p(x) - p(c)) / (x - c)

        Actual differentation P'(k) (x) = P'(k) (c) / h^k
    """

    n = len(a) - 1
    b = [0.0] * (n + 1)
    bc_values = [""] * (n + 1)

    b[n] = a[n]
    for i in range(n - 1, -1, -1):
        b[i] = a[i] + c * b[i + 1]
        bc_values[i + 1] = b[i + 1] * c

    # Prepare table (i from n to 0)
    df = pd.DataFrame({
        "i": list(range(n, -1, -1)),
        "a_i": [a[i] for i in range(n, -1, -1)],
        "b_i*c": [bc_values[i] for i in range(n, -1, -1)],
        "b_i = a_i + b_(i+1)*c": [b[i] for i in range(n, -1, -1)]
    })

    p_c = b[0]
    q_coeff = b[1:]
    return df, p_c, q_coeff, b

def all_derivatives(a, c, gap):
    """
    Compute all derivatives p^(i)(c) using repeated Horner division
    and display in transposed table format.
    """
    coeffs = a.copy()
    degree = len(a) - 1
    results = []
    b0_list = []
    derivative_list = []
    actual_dev_list = []

    # Perform repeated synthetic division
    for i in range(degree + 1):
        df, b0, next_coeff, b_all = synthetic_division(coeffs, c)
        results.append(b_all)
        b0_list.append(b0)
        derivative_list.append(b0 * math.factorial(i))
        actual_dev_list.append(b0 * math.factorial(i) / gap**i)
        coeffs = next_coeff
        if len(coeffs) == 0:
            break

    # Pad b_i lists for equal column length
    max_len = max(len(b) for b in results)
    for b in results:
        b.extend([None] * (max_len - len(b)))

    # Create DataFrame horizontally
    df = pd.DataFrame(results).T
    df.columns = [f"i={i}" for i in range(len(results))]

    # Insert first column for original a coefficients
    a_col = a + [None] * (df.shape[0] - len(a))
    df.insert(0, "a_i", a_col)

    # Add b_0 and p^(i)(c) rows
    df.loc["b_0"] = [None] + b0_list
    df.loc["p^(i)(c)"] = [None] + derivative_list
    df.loc["p^(i)(x)"] = [None] + actual_dev_list

    # Add a row on top showing the value of c
    df.loc["c"] = [c] + [None] * (df.shape[1] - 1)
    df = df.loc[["c"] + [idx for idx in df.index if idx != "c"]]  # Move row to top

    return df

In [64]:
coeff_list = final_coeff_df['Coeff'].tolist()

x_val = 2.55
t_val = (x_val - x0_val) / h

print(f"x0_val: {x0_val}; gap: {h}")

df2 = all_derivatives(coeff_list, t_val, h)
df2.style

x0_val: 2.5525; gap: 0.11500000000000021


,a_i,i=0,i=1,i=2,i=3,i=4,i=5,i=6,i=7
c,-0.021739,nan,nan,nan,nan,nan,nan,nan,nan
0,0.700600,0.703305,-0.124721,0.014195,-0.003096,-0.001462,0.000549,0.000118,-0.000035
1,-0.124108,-0.124414,0.014127,-0.003127,-0.001450,0.000551,0.000117,-0.000035,nan
2,0.013989,0.014058,-0.003159,-0.001438,0.000554,0.000117,-0.000035,nan,nan
3,-0.003220,-0.003190,-0.001426,0.000556,0.000116,-0.000035,nan,nan,nan
4,-0.001401,-0.001414,0.000559,0.000115,-0.000035,nan,nan,nan,nan
5,0.000564,0.000561,0.000114,-0.000035,nan,nan,nan,nan,nan
6,0.000113,0.000114,-0.000035,nan,nan,nan,nan,nan,nan
7,-0.000035,-0.000035,nan,nan,nan,nan,nan,nan,nan
b_0,nan,0.703305,-0.124721,0.014195,-0.003096,-0.001462,0.000549,0.000118,-0.000035
